## Example of usage of the Julia interface to the circuit simulator PAN

In [ ]:
using PAN
using Printf: @printf
using Plots

The purpose of this notebook is to show some of PAN's basic functionalities that can be accessed by means of the Julia interface.

We will use a netlist that implements the Van der Pol oscillator, whose equations are the following:

$$
\begin{equation*}
\left\{
\begin{array}{rcl}
\dot{x} & = & \epsilon\ (1 - x^2)\ x - y \\
\dot{y} & = & x \ ,
\end{array}
\right.
\end{equation*}
$$

where $x$ represents the voltage across the capacitor, $y$ is the current in the inductor and the parameter $\epsilon$ is a positive constant that determines the frequency of the oscillator. If $\epsilon \ll 1$, the oscillating frequency is equal to $2\pi$, whereas if $\epsilon$ is larger, the frequency is a function of $\epsilon$ itself.

We will start with $\epsilon=0.001$ and then increase it to 5.

The following netlist defines the Van der Pol circuit in PAN.

In [ ]:
netlist = """
ground electrical gnd

parameter EPS=1m

; This dummy alter is necessary to be able to change the value of the parameter from Julia
Alrt alter param="EPS" rt=yes

; The circuit

p1    x   gnd  x    vccs func=-EPS*v(x) + EPS*v(x)^3
c1    x   gnd  capacitor c=1 icon=2m
l1    x   gnd  inductor  l=1
""";

Write the above netlist to file.

In [ ]:
netlist_file = "vanderpol.pan"
open(netlist_file, "w") do io
    write(io, netlist)
end;

### Load the netlist with the description of the circuit
No analysis is performed at this point, since none is defined in the netlist.

In [ ]:
ok, libs = load_netlist(netlist_file);
if ! ok
    @printf("Problems loading netlist '%s'.\n", netlist_file)
end

### Run a first (short) transient analysis

In [ ]:
tstop = 1000
mem_vars = ["time", "x", "l1:i"]
tran_1 = tran("Tr1", tstop, mem_vars, libs, uic=1, vabstol=1e-10);

### Run a longer envelope analysis
The dynamics of the system are very slow, hence it is convenient to use an envelope analysis to speed up the integration of the system.

In [ ]:
tstop = 50e3
T = 2 * pi
env = envelope("En", tstop, T, mem_vars, libs, autonomous="yes", restart="no");

### Run a shooting analysis
We can confirm that the envelope integration has reached the steady-state by comparing the last cycle obtained with the envelope with the one found by means of the shooting procedure.

In [ ]:
shoot_1 = shooting("Sh1", T, mem_vars, libs, autonomous="yes", restart="no",
    method=2, floquet="yes", tmax=100e-6, printmo="yes", eigf=1, eigfnorm=1, samples=200);

### Alter the value of a parameter
We increase the value of $\epsilon$ from 0.001 to 5.

In [ ]:
alter("Al", "EPS", 5, libs, invalidate="no", annotate=4);

### Run a second shooting analysis
We want to see how much the shape of the limit cycle has changed with the new value of $\epsilon$.

In [ ]:
shoot_2 = shooting("Sh2", T, mem_vars, libs, autonomous="yes", restart="no",
    method=2, floquet="yes", tmax=100e-6, printmo="yes", eigf=1, eigfnorm=1, samples=200);

### Plot the results

In [ ]:
gr()
l = @layout [a b ; c d]

p1 = plot(tran_1[:,1], tran_1[:,2], color="black", lw=1, label="Tran");
plot!(env[:,1], env[:,2], color="red", lw=1, label="Envelope");
plot!(shoot_1[:,1], shoot_1[:,2], color="green", lw=1, label="Shooting-1");
plot!(shoot_2[:,1], shoot_2[:,2], color="magenta", lw=1, label="Shooting-2");
xlabel!("Time [s]");
ylabel!("v_C [V]");

p2 = plot(shoot_1[:,2], shoot_1[:,3], color="green", lw=2, label="Shooting-1");
plot!(shoot_2[:,2], shoot_2[:,3], color="magenta", lw=2, label="Shooting-2");
xlabel!("v_C [V]");
ylabel!("i_L [A]");

idx = findall(t -> t > 950, tran_1[:,1]);
p3 = plot(tran_1[idx,1], tran_1[idx,2], color="black", lw=1, legend=false);
idx = findall(t -> t < 1100, env[:,1]);
plot!(env[idx,1], env[idx,2], color="red", lw=1);
xlabel!("Time [s]");
ylabel!("v_C [V]");

idx = findall(t -> t > tstop - 20, env[:,1])
p4 = plot(env[idx,1], env[idx,2], color="red", lw=2, legend=false);
plot!(shoot_1[:,1], shoot_1[:,2], color="green", lw=2);
plot!(shoot_2[:,1], shoot_2[:,2], color="magenta", lw=2);
xlabel!("Time [s]");
ylabel!("v_C [V]");

plot(p1, p2, p3, p4, layout=l, show=true)